In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform
import warnings

# seaborn 사용하기
import seaborn as sns

plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

warnings.filterwarnings('ignore')

In [8]:
food = pd.read_csv('./Data/seogyodong+korean-ratio.csv')
food.head()

,기준_년_코드,기준_분기_코드,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,점포수,한식음식점여부
0,2021,1,1000466,동교로17길,CS100001,한식음식점,243935896,14,1
1,2021,1,1000466,동교로17길,CS100002,중식음식점,45393178,1,0
2,2021,1,1000466,동교로17길,CS100003,일식음식점,55589434,3,0
3,2021,1,1000466,동교로17길,CS100006,패스트푸드점,9925407,3,0
4,2021,1,1000466,동교로17길,CS100008,분식전문점,40136157,6,0


### 붙일 데이터들 확인하기

In [9]:
# 집객시설 데이터 : 상권별로 있다. (2015-2021)
pd.read_csv('./Raw/서울시_우리마을가게_상권분석서비스(구_상권_집객시설).csv',encoding='euc-kr')

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,집객시설_수,관공서_수,은행_수,종합병원_수,일반_병원_수,...,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수,기준_분기_코드
0,2021,A,골목상권,1000862,방배로18길,8,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3
1,2021,A,골목상권,1000611,개봉로17다길,17,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,3
2,2021,A,골목상권,1000606,화곡로44나길,11,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3
3,2021,A,골목상권,1000602,초원로16길,5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3
4,2021,A,골목상권,1000601,초록마을로8길,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40216,2015,U,관광특구,1001494,종로·청계 관광특구,212,NaN,30.0,NaN,NaN,...,NaN,NaN,22.0,6.0,NaN,NaN,NaN,5.0,46.0,2
40217,2015,U,관광특구,1001496,강남 마이스 관광특구,51,NaN,8.0,NaN,NaN,...,1.0,NaN,15.0,2.0,NaN,NaN,NaN,1.0,14.0,2
40218,2015,U,관광특구,1001495,잠실 관광특구,102,NaN,11.0,NaN,NaN,...,1.0,NaN,19.0,7.0,NaN,NaN,NaN,NaN,18.0,2
40219,2015,U,관광특구,1001496,강남 마이스 관광특구,51,NaN,8.0,NaN,NaN,...,1.0,NaN,15.0,2.0,NaN,NaN,NaN,1.0,14.0,3


In [10]:
# 직장인구 수 데이터 : 상권별로 있다. (2014-2021)
pd.read_csv('./Raw/서울시_우리마을가게_상권분석서비스(구_상권_직장인구).csv',encoding='euc-kr')

,기준_년월_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,연령대_10_직장_인구_수,...,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
0,2021,1,A,골목상권,1000007,북촌로5길,503,419,84,0,...,150,133,88,29,0,12,27,23,14,8
1,2021,1,A,골목상권,1000756,사당로8길,131,70,61,0,...,9,24,22,13,0,14,10,17,15,5
2,2021,1,R,전통시장,1001278,남평화시장,5,3,2,0,...,2,0,0,1,0,0,2,0,0,0
3,2021,1,A,골목상권,1000058,백범로90길,535,288,247,0,...,71,89,57,43,0,34,62,72,55,24
4,2021,1,A,골목상권,1000059,서빙고로71길,432,265,167,1,...,49,58,64,80,1,26,55,59,17,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43227,2014,4,A,골목상권,1000673,시흥대로141길,667,410,257,6,...,142,103,78,22,1,73,56,51,61,15
43228,2014,4,A,골목상권,1000496,와우산로3길,177,105,72,2,...,38,37,12,3,1,19,30,10,12,0
43229,2014,4,A,골목상권,1000705,도신로60길,87,14,73,0,...,2,7,0,5,0,5,7,7,18,36
43230,2014,4,D,발달상권,1001255,서울 은평구 연신내역_3,550,174,376,1,...,51,50,31,23,1,106,99,78,56,36


In [11]:
# 디테일한 점포 수 데이터 : 업종 별로 있다.
pd.read_csv('./Raw/서울시_우리마을가게_상권분석서비스(상권-점포)_2021년.csv',encoding='euc-kr')

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
0,2021,1,A,골목상권,1000001,계동길,CS100001,한식음식점,16,16,0,0,0,0,0
1,2021,1,A,골목상권,1000001,계동길,CS100002,중식음식점,0,1,0,0,0,0,1
2,2021,1,A,골목상권,1000001,계동길,CS100003,일식음식점,2,2,0,0,0,0,0
3,2021,1,A,골목상권,1000001,계동길,CS100004,양식음식점,3,3,0,0,0,0,0
4,2021,1,A,골목상권,1000001,계동길,CS100005,제과점,3,4,25,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347352,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300036,조명용품,14,14,0,0,0,0,0
347353,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300038,자동차부품,10,10,0,0,0,0,0
347354,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300041,예술품,10,10,0,0,0,0,0
347355,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300042,주유소,1,1,0,0,0,0,0


In [12]:
# 아파트 데이터 : 상권 별로 (2014-2021)
pd.read_csv('./Raw/서울시_우리마을가게_상권분석서비스(구_상권_아파트).csv',encoding='euc-kr')

,기준_년_코드,상권_구분_코드,상권_구분_코드_명,아파트_단지_수,아파트_면적_66_제곱미터_미만_세대_수,아파트_면적_66_제곱미터_세대_수,아파트_면적_99_제곱미터_세대_수,아파트_면적_132_제곱미터_세대_수,아파트_면적_165_제곱미터_세대_수,아파트_가격_1_억_미만_세대_수,...,아파트_가격_2_억_세대_수,아파트_가격_3_억_세대_수,아파트_가격_4_억_세대_수,아파트_가격_5_억_세대_수,아파트_가격_6_억_이상_세대_수,아파트_평균_면적,아파트_평균_시가,기준_분기_코드,상권_코드,상권_코드_명
0,2021,A,골목상권,1,5.0,1.0,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,53,148833333,1,1000271,동소문로2길
1,2021,A,골목상권,93,566.0,158.0,3.0,22.0,NaN,17.0,...,364.0,30.0,11.0,12.0,NaN,54,216887087,4,1000147,뚝섬로54길
2,2021,A,골목상권,2,NaN,1.0,3.0,1.0,4.0,NaN,...,NaN,3.0,2.0,NaN,4.0,131,468714286,3,1000007,북촌로5길
3,2021,A,골목상권,5,10.0,7.0,3.0,NaN,6.0,3.0,...,5.0,2.0,NaN,1.0,6.0,118,444225714,3,1000008,북촌로5나길
4,2021,A,골목상권,12,95.0,23.0,2.0,NaN,NaN,34.0,...,44.0,2.0,NaN,NaN,NaN,49,157061533,3,1000010,성균관로5길
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41363,2014,U,관광특구,19,355.0,198.0,237.0,643.0,192.0,56.0,...,109.0,131.0,2.0,123.0,994.0,98,423657589,3,1001495,잠실 관광특구
41364,2014,U,관광특구,72,344.0,200.0,59.0,64.0,61.0,65.0,...,202.0,161.0,82.0,30.0,50.0,82,307303484,2,1001491,이태원 관광특구
41365,2014,U,관광특구,5,32.0,74.0,13.0,NaN,NaN,7.0,...,24.0,10.0,NaN,NaN,NaN,83,129443464,2,1001493,동대문패션타운 관광특구
41366,2014,U,관광특구,3,266.0,10.0,NaN,NaN,NaN,135.0,...,NaN,NaN,NaN,NaN,NaN,41,88804481,2,1001494,종로·청계 관광특구


### 상권 배후지와 상권 
- 상권 배후지는 그 상권을 이용하는 고객들의 영역을 말하는 것이다. <br><br>
- 상권은 그 지역만을 말한다. <br><br>
- 예비 창업주를 대상으로 하기 때문에 배후지 파악까지는 힘들 것이라 생각하여 상권 데이터만으로 feature를 구성하려고 한다. <br><br>

### 확인해 볼만한 데이터 
- 아파트 데이터 : 아파트 평균 면적 데이터 , 아파트 단지수 ... <br><br>
- 점포수 데이터 : 프랜차이즈 점포수 데이터 <br><br>

### 나머지 데이터들로 가능한것??
- 직장인 데이터 , 상주인구 데이터, 생활인구 데이터로는 어디가 매출에 큰 비중을 차지하는지를 알아볼 수 있을 것 같다. <br><br>
- 집객시설 데이터로는 백화점이나 큰 대형마켓이 골목상권의 매출을 올려주는지 아닌지 와 상권 매출에 직접적으로 이득이 되는 집객시설이 있는지를 파악할 수 있을 것 같다. <br><br>